## **59.Scraping Superbalist**

In [ ]:
import requests
from bs4 import BeautifulSoup
import schedule, time, os, smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 

url = "https://superbalist.com/men/accessories/headwear/u-aero-dfadv-tlwnd-elt-cap-black/766719?clickRef=search_listing"
my_price = 450

def scrape():

  response = requests.get(url)
  html = response.text
  soup = BeautifulSoup(html, 'html.parser')
  
  # Find the first span with class "price" and get its text
  price_tag = soup.find("span", {"class": "price"})
  price_text = price_tag.get_text(strip=True)  # Get text and remove whitespace
  
  current_price = int(price_text[1:])

  # Initialise empty dictionary
  price_dict = {}
  
  price_dict[time.time()] = {"current_price": current_price, "my_price": my_price}

  for key, value in price_dict.items():
    if value["current_price"] < value["my_price"]:
      sendMail(value["current_price"], value["my_price"])
    else:
      noMail(value["current_price"], value["my_price"])

# When you use the parameter names (current_price and my_price) within the sendMail() function, they represent the values that were passed to the function when it was called.
# The actual names are not important, you could also write def sendMail(a,b), where a and b would refer to value["current_price"] and value["my_price"] respectively.
# The arguments you pass are associated based on their positions, not their names.
def sendMail(current_price, my_price):
  password = os.environ["mailPassword"]
  username = os.environ["mailUsername"]
  
  email = f"<p><a href={url}>This item</a> is now {current_price} which is below your purchase level of {my_price}</p>"
  server = "smtp.gmail.com"
  port = 587
  s = smtplib.SMTP(host = server, port = port)
  s.starttls()
  s.login(username, password)

  msg = MIMEMultipart()
  msg['To'] = "karl.meier.mattern@pwc.com"
  msg['From'] = username
  msg['Subject'] = "Price update"
  msg.attach(MIMEText(email, 'html'))
  s.send_message(msg)
  del msg

def noMail(current_price, my_price):
  password = os.environ["mailPassword"]
  username = os.environ["mailUsername"]
  
  email = f"<p><a href={url}>This item</a> is currently {current_price}, which is above your purchase level of {my_price}</p>"
  server = "smtp.gmail.com"
  port = 587
  s = smtplib.SMTP(host = server, port = port)
  s.starttls()
  s.login(username, password)

  msg = MIMEMultipart()
  msg['To'] = "karl.meier.mattern@pwc.com"
  msg['From'] = username
  msg['Subject'] = "Price update"
  msg.attach(MIMEText(email, 'html'))
  s.send_message(msg)
  del msg
  

schedule.every(10).seconds.do(printMe)

def printMe():
  print("🪓🪓🪓 Scraping")
  scrape()

# the While True code is intended to keep the program running indefinitely and execute the scheduled tasks managed by the schedule module.
# schedule.run_pending() and time.sleep(1) is necessary to continuously check for scheduled tasks and execute them according to the schedule you've defined.
# By setting the sleep interval to match the schedule interval, you ensure that the program runs efficiently and aligns with your intended schedule.
while True:
  schedule.run_pending()
  time.sleep(10)